In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt

from sklearn.naive_bayes import BernoulliNB
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_    
%matplotlib inline

In [2]:
data = pd.read_csv("data.csv") #补货数据

In [15]:
useful_col = ['AreaName','ShopCode','MatCode','SizeName','LastWeekSaleQty',"AvgLastWeekSaleQty",
'LastWeekMoveQty','LastWeekInitQty','CZPFlag','SKCCzpFlag','CZPSuppleRate','DisPlayQty','SizeCoreFlag']

In [16]:
df = data[useful_col]

In [5]:
df["LastWeekSaleQty"].isna().value_counts()

False    74982
True     55728
Name: LastWeekSaleQty, dtype: int64

In [9]:
standard = pd.read_excel("商品指标标准.xlsx")

In [10]:
standard.to_csv("standard.csv",index =False)

In [53]:
(df["LastWeekSaleQty"]==(df["AvgLastWeekSaleQty"]*7).apply(round)).value_counts()

True     69792
False    60918
dtype: int64

In [54]:
(data["LastWeekSaleQty"]==(data["AvgLastWeekSaleQty"]*7).apply(round)).value_counts()

True     69792
False    60918
dtype: int64

In [59]:
data["LastWeekSaleQty"][data["LastWeekSaleQty"]!=(data["AvgLastWeekSaleQty"]*7).apply(round)].value_counts().sum()

5190

In [6]:
df["LastWeekSaleQty"]=((data["AvgLastWeekSaleQty"]*7).apply(round))

/home/gang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
df.head()

,AreaName,ShopCode,MatCode,SizeName,LastWeekSaleQty,AvgLastWeekSaleQty,LastWeekMoveQty,LastWeekInitQty,CZPFlag,SKCCzpFlag,DisPlayQty,SizeCoreFlag
0,邯郸市,132214J,M3ZT202148Y0,L,1,0.14,NaN,1,爆,畅,2,1
1,邯郸市,132214J,M3ZT202148Y0,XL,1,0.14,NaN,1,爆,畅,2,1
2,邯郸市,132214J,M3ZT202148Y0,XXL,1,0.14,NaN,7,滞,畅,2,1
3,大同市,141208J,M2D1900136H0,L,2,0.29,NaN,5,爆,平,2,1
4,大同市,141208J,M2D1900136H0,XL,0,0.00,NaN,2,超滞,平,2,1


In [11]:
standard.head()

,分公司,店铺代码,店铺名称,店铺分档,店铺类型,陈列SKC标准,满场率标准,齐码率,动销率标准,库存均深标准,在店库存件数 （下限值）
0,华南,01060,黄埔大沙地店,折扣店,D2,290,100%-150%,0.440000,0.400000,12.931370,3750
1,华南,01061,花都建设路店,重点店,D2,193,90%-120%,0.761905,0.400000,12.931370,2246
2,华南,01063,东门步行街1085号店,重点店,A2,242,90%-130%,0.761905,0.610975,19.425092,4231
3,华南,01065,北京路一店,重点店,A1,390,90%-130%,0.761905,0.624922,17.235936,6050
4,华南,01069,十甫路161号店,重点店,B2,276,90%-130%,0.761905,0.500000,14.350595,3565


In [13]:
(data["MatCode"].value_counts()==data["MatID"].value_counts()).value_counts()

True    2152
dtype: int64

In [12]:
data["ShopCode"].value_counts().size

311

In [78]:
data["ItemName"].isna().value_counts()

True    130710
Name: ItemName, dtype: int64

In [22]:
(data["FirstArriveDate"]==data["FirstInStockDate"]).value_counts()

True    130710
dtype: int64

In [65]:
(data["LastWeekInitQty"]==data["WeekInitQty"]).value_counts()

True    130710
dtype: int64

In [93]:
df.groupby(by = ["ShopCode","MatCode","SizeID"],sort = False)["SizeCoreFlag"].apply(lambda x:x.value_counts().shape).value_counts()

(1,)    87881
Name: SizeCoreFlag, dtype: int64

In [76]:
df.groupby(by = ["ShopCode","MatCode","SizeID"],sort = False).size().value_counts()

1    63223
3    18171
2     6487
dtype: int64

**统计店铺内skc数量**
我们发现了好像很多店铺都只有几十款，也就是貌似压根不满足满场率

In [101]:
df.groupby(by = "ShopCode",sort = False)["MatID"].apply(lambda x:x.value_counts().shape).value_counts()

(73,)     8
(85,)     8
(68,)     7
(71,)     7
(83,)     6
(75,)     6
(49,)     5
(56,)     5
(99,)     5
(37,)     5
(50,)     5
(88,)     5
(55,)     5
(118,)    5
(63,)     4
(76,)     4
(93,)     4
(57,)     4
(84,)     4
(80,)     4
(105,)    4
(161,)    4
(87,)     4
(66,)     4
(103,)    4
(86,)     4
(79,)     4
(59,)     4
(115,)    4
(94,)     4
         ..
(169,)    1
(124,)    1
(44,)     1
(190,)    1
(110,)    1
(17,)     1
(133,)    1
(286,)    1
(21,)     1
(222,)    1
(34,)     1
(92,)     1
(126,)    1
(27,)     1
(25,)     1
(243,)    1
(231,)    1
(157,)    1
(167,)    1
(113,)    1
(114,)    1
(195,)    1
(152,)    1
(100,)    1
(8,)      1
(181,)    1
(121,)    1
(153,)    1
(203,)    1
(132,)    1
Name: MatID, Length: 124, dtype: int64

**统计了在店库存**
即使将三天的全部算上可能会导致重复计入，可以看到貌似库存量也基本不满足在店库存的下限

In [122]:
df.groupby(by = "ShopCode", sort = False)["LastWeekInitQty"].sum()

ShopCode
132214J     213
141208J    1010
131223J     401
442218J    1148
132215J     242
441211J    1249
141211J     643
141213J    1261
132225J     181
1040       1059
1023       1536
1013        810
1028       1224
134217J     125
142221J     583
651203J     200
132218L     194
141215L    1273
5010        867
1024       1413
442219J     105
142222J    1021
5014       1331
5015       1232
5002       1059
5005        613
131226J     497
442225J     393
442212J    1102
442214J     598
           ... 
353007J     718
423190J     438
411041J     569
431199Z    2471
411042J     249
411044J     356
421191J     332
653139J     610
531006J     956
421192J     358
511198J       0
511199J     301
511200J     285
411054J     180
511202J     279
411057J     594
511203J     305
433202J     314
113131J     560
131139J     720
131208J     200
122203J     161
133113J     330
131114J     311
461159J     733
121004J     192
123001J     577
133106J     535
463014J     342
463012J     710
Name: LastWeekI

In [72]:
(data["AvgLastWeekSaleQty"]*7).apply(round).value_counts()

 0     56234
 1     46682
 2     17582
 3      4696
 4      2801
 5       966
 7       787
 6       416
 8       125
 9        84
 10       74
 14       63
-1        53
 12       30
 11       24
 13       21
 15       14
 16       11
 21        9
 19        7
 28        6
 17        5
 18        5
 23        3
 22        2
 24        2
-2         1
 25        1
 26        1
 30        1
 31        1
 32        1
 33        1
 35        1
Name: AvgLastWeekSaleQty, dtype: int64

In [45]:
data["AvgLastWeekSaleQty"].isna().value_counts()

False    130710
Name: AvgLastWeekSaleQty, dtype: int64

In [62]:
df["ValidWeekAvg"].head()

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: ValidWeekAvg, dtype: float64

In [20]:
data["FirstArriveDate"].head()

0    2019-03-29 00:00:00
1    2019-03-29 00:00:00
2    2019-03-29 00:00:00
3    2019-03-21 00:00:00
4    2019-03-21 00:00:00
Name: FirstArriveDate, dtype: object

In [119]:
data["SaleBgDate"].head()

0    2019-06-03 00:00:00
1    2019-06-03 00:00:00
2    2019-06-03 00:00:00
3    2019-06-02 00:00:00
4    2019-06-02 00:00:00
Name: SaleBgDate, dtype: object

In [14]:
data["MatID"].value_counts().head()

M2X1904171L0    1433
M1XT100889W0    1271
M2X1904385B1    1025
M2X1903986F1     909
M2X1903986H9     890
Name: MatID, dtype: int64

In [15]:
data["CZPFlag"].head()

0     爆
1     爆
2     滞
3     爆
4    超滞
Name: CZPFlag, dtype: object

In [17]:
data["CZPSuppleRate"].value_counts()

2.0    49221
0.0    37998
1.0    24820
1.5    18671
Name: CZPSuppleRate, dtype: int64

In [86]:
data["DisPlayQty"].value_counts()

2    74600
0    36665
1    19445
Name: DisPlayQty, dtype: int64

## 调配

** 在这里我们认为单店单sku在数据集中只出现一次，如果不适用再修改**

### 计算供需
SKC为 滞销 超滞销的 可以 供出
SKC为 平， 其 SKU 为滞销 超滞销的，可以供出

SKC 为 畅销 爆款 可以 提出需求
SKC 为 平， 其SKU 为 畅销 爆款的， 可以提出需求
 
需求= 上周销量 * 畅滞平系数 + 陈列需求 – 现有库存

陈列需求  = if(核心尺码) X = 2
 			Else X = 1

In [4]:
df1 =pd.read_csv("库存信息.csv",encoding="gbk")

In [5]:
df1.head()

,OperationDate,StockCode,StockName,StockOwerShip,matid,SizeName,StockAmt,StockQty,PathStockAmt,PathStockQty,YearNo,SeasonName,OrderAttribName,CategoryName,MidCategoryName,SmallCategoryName
0,2019-06-11,01023,朔州北关一店,华北分公司,M2X1904356W0,XL,99.9,1,0.0,0,2019,夏,第5波段,服装,T恤,短袖T恤
1,2019-06-11,01023,朔州北关一店,华北分公司,M3X18H3001K0,F,50.0,2,0.0,0,2017,夏,第三波段,配件,服饰配件,腰带
2,2019-06-11,01023,朔州北关一店,华北分公司,M2X1903960L0,33,796.0,4,0.0,0,2019,夏,第三波段,服装,裤装,休闲九分裤
3,2019-06-11,01023,朔州北关一店,华北分公司,M2X1904138B0,XXL,129.0,1,258.0,2,2019,夏,第4波段,服装,T恤,短袖T恤
4,2019-06-11,01023,朔州北关一店,华北分公司,M2X1903983W0,XXL,447.0,3,0.0,0,2019,夏,第三波段,服装,T恤,短袖T恤


In [19]:
df =pd.read_csv("补货信息.csv",encoding="gbk")

In [20]:
df.head()

,ExtrSource,CompanyNo,StockDate,FHStockName,FHStockID,AreaName,StockID,ShopCode,ShopName,MatID,...,CompanyID,MaterialID,AvgWeekSaleQty,SaleDays,WeekSaleQty,WeekMoveQty,WeekInitQty,FirstArriveDate,MaterialShortName,MaterialCode
0,KM,KM,2019-06-11,太原分仓,KM000094,朔州市,KM000002,01023,朔州北关一店,3B60001B2,...,KM,3B60001B2,0.14,794,1.0,NaN,4,2017-04-08,包,3B60001B2
1,KM,KM,2019-06-11,太原分仓,KM000094,朔州市,KM000002,01023,朔州北关一店,3C10001W0,...,KM,3C10001W0,0.00,786,NaN,NaN,47,2017-04-16,水杯,3C10001W0
2,KM,KM,2019-06-11,太原分仓,KM000094,朔州市,KM000002,01023,朔州北关一店,3J10002B0,...,KM,3J10002B0,0.00,794,NaN,NaN,19,2017-04-08,眼镜,3J10002B0
3,KM,KM,2019-06-11,太原分仓,KM000094,朔州市,KM000002,01023,朔州北关一店,3J10003S0,...,KM,3J10003S0,0.14,773,1.0,NaN,1,2017-04-29,眼镜,3J10003S0
4,KM,KM,2019-06-11,太原分仓,KM000094,朔州市,KM000002,01023,朔州北关一店,3JP11001R3,...,KM,3JP11001R3,0.00,816,NaN,NaN,0,2017-03-17,腰带,3JP11001R3


In [22]:
df["LastWeekSaleQty"]=((df["AvgLastWeekSaleQty"]*7).apply(round))

In [23]:
useful_col = ['AreaName','ShopCode','MatCode','SizeName','LastWeekSaleQty',"AvgLastWeekSaleQty",
'LastWeekMoveQty','LastWeekInitQty','CZPFlag','SKCCzpFlag','CZPSuppleRate','DisPlayQty','SizeCoreFlag']
df = df[useful_col]

In [24]:
df['LastWeekMoveQty'].fillna(0,inplace = True)

In [25]:
storage = df['LastWeekInitQty'] + df['LastWeekMoveQty'] - df['LastWeekSaleQty']
total_need = df['LastWeekSaleQty'] * df['CZPSuppleRate'] + df['DisPlayQty'] #需求= 上周销量 * 畅滞平系数 + 陈列需求 – 现有库存

In [26]:
df["Qty"] = storage

In [27]:
df['SKCCzpFlag'].value_counts()

超滞    446837
滞     131409
平      72594
爆      34416
畅      30706
Name: SKCCzpFlag, dtype: int64

In [28]:
df.columns

Index(['AreaName', 'ShopCode', 'MatCode', 'SizeName', 'LastWeekSaleQty',
       'AvgLastWeekSaleQty', 'LastWeekMoveQty', 'LastWeekInitQty', 'CZPFlag',
       'SKCCzpFlag', 'CZPSuppleRate', 'DisPlayQty', 'SizeCoreFlag', 'Qty'],
      dtype='object')

In [29]:
#计算需求减去库存
need = total_need-storage
#考虑Sku & Skc的CZP
def czp(df_):
    skc = df_["SKCCzpFlag"]
    sku = df_["CZPFlag"]
    if(skc in ["滞","超滞"]):
        return -1
    elif(skc =="平"):
        if(sku in ["滞","超滞"]):
            return -1
    if(skc in ["畅","爆"]):
        return 1
    elif(skc =="平"):
        if(sku in ["畅","爆"]):
            return 1
    return 0    
    
    
        
#filt = df[""] 

In [30]:
czp_coefficient =df.apply(czp,axis =1)#SKC为 滞销 超滞销的 可以 供出 SKC为 平， 其 SKU 为滞销 超滞销的，可以供出

                                      # SKC 为 畅销 爆款 可以 提出需求 SKC 为 平， 其SKU 为 畅销 爆款的， 可以提出需求

In [31]:
#need_coefficient定义为 need: 1, provide :-1

need_coefficient = ((need>0).astype("int"))*2-1

In [33]:
need_coefficient.value_counts()

-1    569429
 1    146533
dtype: int64

In [34]:
need_sku = need*czp_coefficient 
need_sku= need_sku+abs(need_sku)#只保留大于零的，表示供需consistent
need_sku = need_sku*need_coefficient #赋予符号

In [35]:
df["SkuNeed"] = need_sku

In [36]:
df["need_coefficient"]=need_coefficient

In [37]:
df.to_csv("need_611.csv",index = False)

In [38]:
df = pd.read_csv("need_611.csv")

In [39]:
df.head()

,AreaName,ShopCode,MatCode,SizeName,LastWeekSaleQty,AvgLastWeekSaleQty,LastWeekMoveQty,LastWeekInitQty,CZPFlag,SKCCzpFlag,CZPSuppleRate,DisPlayQty,SizeCoreFlag,Qty,SkuNeed,need_coefficient
0,朔州市,01023,3B60001B2,JM,1,0.14,0.0,4,平,平,1.0,0,0,3.0,-0.0,-1
1,朔州市,01023,3C10001W0,JM,0,0.00,0.0,47,超滞,超滞,0.0,0,0,47.0,-94.0,-1
2,朔州市,01023,3J10002B0,JM,0,0.00,0.0,19,超滞,超滞,0.0,0,0,19.0,-38.0,-1
3,朔州市,01023,3J10003S0,JM,1,0.14,0.0,1,爆,爆,2.0,0,0,0.0,4.0,1
4,朔州市,01023,3JP11001R3,M,0,0.00,0.0,0,超滞,超滞,0.0,0,0,0.0,-0.0,-1


### 调配的保护机制
根据开发思路，需要供出的货品经过清洗有以下 种结果：
1. 由于2.3(库存量保护) 2.5(调入不调出保护) 2.7(top保护) 2.8(最小包裹保护)。 不调出
2. 由于2.6(尾数清零保护),整款（skc）调出
3. 正常（sku）调出

PS:齐码率的保护貌似不起作用，因为如果断码那么由于尾数清零保护整款会调出。  解决方案：调出时优先调出调后齐码的款，而尽量不调出调后缺码的款
以下的解决方案基于上面的所有假设

In [40]:
df["SizeCoreFlag"].value_counts() #SizeCoreFlag 代表是否是核心尺码

0    422973
1    292989
Name: SizeCoreFlag, dtype: int64

In [41]:
standard = pd.read_csv("standard.csv")

In [42]:
standard.head()

,分公司,店铺代码,店铺名称,店铺分档,店铺类型,陈列SKC标准,满场率标准,齐码率,动销率标准,库存均深标准,在店库存件数 （下限值）
0,华南,01060,黄埔大沙地店,折扣店,D2,290,100%-150%,0.440000,0.400000,12.931370,3750
1,华南,01061,花都建设路店,重点店,D2,193,90%-120%,0.761905,0.400000,12.931370,2246
2,华南,01063,东门步行街1085号店,重点店,A2,242,90%-130%,0.761905,0.610975,19.425092,4231
3,华南,01065,北京路一店,重点店,A1,390,90%-130%,0.761905,0.624922,17.235936,6050
4,华南,01069,十甫路161号店,重点店,B2,276,90%-130%,0.761905,0.500000,14.350595,3565


In [43]:
standard.columns

Index(['分公司', '店铺代码', '店铺名称', '店铺分档', '店铺类型', '陈列SKC标准', '满场率标准', '齐码率',
       '动销率标准', '库存均深标准', '在店库存件数\n（下限值）'],
      dtype='object')

In [44]:
standard.rename({"店铺代码":"ShopCode","在店库存件数\n（下限值）":"StorageMin","陈列SKC标准":"SkcNum","齐码率":"UniformRate"},axis = 1,inplace =True)

In [45]:
df_ = pd.merge(df,standard,on ="ShopCode")

In [125]:
group.apply(lambda x:x)  #发现了一种将groupby转换成df的方法

,AreaName,ShopCode,MatCode,SizeName,LastWeekSaleQty,AvgLastWeekSaleQty,LastWeekMoveQty,LastWeekInitQty,CZPFlag,SKCCzpFlag,...,分公司,店铺名称,店铺分档,店铺类型,SkcNum,满场率标准,UniformRate,动销率标准,库存均深标准,StorageMin
0,邯郸市,132214J,M3ZT202148Y0,L,1,0.14,0.0,1,爆,畅,...,华北,邯郸峰峰新世纪店中店,非重点店,E2,145,100%-140%,0.625000,0.439235,11.476063,1664
1,邯郸市,132214J,M3ZT202148Y0,XL,1,0.14,0.0,1,爆,畅,...,华北,邯郸峰峰新世纪店中店,非重点店,E2,145,100%-140%,0.625000,0.439235,11.476063,1664
2,邯郸市,132214J,M3ZT202148Y0,XXL,1,0.14,0.0,7,滞,畅,...,华北,邯郸峰峰新世纪店中店,非重点店,E2,145,100%-140%,0.625000,0.439235,11.476063,1664
3,邯郸市,132214J,1X1215010L0,L,1,0.20,7.0,0,平,平,...,华北,邯郸峰峰新世纪店中店,非重点店,E2,145,100%-140%,0.625000,0.439235,11.476063,1664
4,邯郸市,132214J,1X1215010L0,XL,0,0.00,0.0,0,超滞,平,...,华北,邯郸峰峰新世纪店中店,非重点店,E2,145,100%-140%,0.625000,0.439235,11.476063,1664
5,邯郸市,132214J,1X1215010L0,XXL,0,0.00,0.0,0,超滞,平,...,华北,邯郸峰峰新世纪店中店,非重点店,E2,145,100%-140%,0.625000,0.439235,11.476063,1664
6,邯郸市,132214J,1X2519002B0,31,0,0.00,3.0,0,超滞,平,...,华北,邯郸峰峰新世纪店中店,非重点店,E2,145,100%-140%,0.625000,0.439235,11.476063,1664
7,邯郸市,132214J,1X2519002B0,32,1,0.20,3.0,0,爆,平,...,华北,邯郸峰峰新世纪店中店,非重点店,E2,145,100%-140%,0.625000,0.439235,11.476063,1664
8,邯郸市,132214J,1X2519002B0,33,0,0.00,0.0,0,超滞,平,...,华北,邯郸峰峰新世纪店中店,非重点店,E2,145,100%-140%,0.625000,0.439235,11.476063,1664
9,邯郸市,132214J,1X2519002B0,34,0,0.00,0.0,0,超滞,平,...,华北,邯郸峰峰新世纪店中店,非重点店,E2,145,100%-140%,0.625000,0.439235,11.476063,1664


In [46]:
#2.5调入不调出保护，第一遍过滤,如果上周有调入则不调出,但是可以调入
mask= (df_["need_coefficient"]<0)
df_["SkuNeed"][mask]=df_["SkuNeed"][mask]*(df_["LastWeekMoveQty"][mask]<=0)
#2.7(top保护) 由于top保护比较简单，可以先用top保护进行一遍过滤
group = df_.groupby(by = "ShopCode",sort = False)
def top(df):
    df_temp = df.sort_values(by = "LastWeekSaleQty",ascending = False)
    if(df.shape[0]>19):
        lst = df_temp.index[:20]
    else:
        lst = df_temp.index
    df["SkuNeed"][lst]  =0   
    return df
df_=group.apply(top)

#2.8(最小包裹保护) 由于最小包裹保护也会改变满场率，放在库存量保护之前
group = df_.groupby(by = "ShopCode",sort = False)
def minpackage(df):
    mask = (abs(df["SkuNeed"])<3)
    df["SkuNeed"][mask]=0
    return df
df_ = group.apply(minpackage)    

#2.6尾数清零保护,整款（skc）调出
group = df_.groupby(by = "ShopCode",sort = False)
def clean(df):
    mask =(((df["Qty"]+df["SkuNeed"])==0)*df["SizeCoreFlag"]*(df["SkuNeed"]<0)==1) #找到导致缺码且要调出的sku
    skc = df["MatCode"][mask] #进而找到缺码的skc的编号
    mask=df["MatCode"].apply(lambda x: True if x in skc.values else False) #把缺码的全部调出,这行代码找到全部调出skc的sku
    df["SkuNeed"][mask] = df["Qty"][mask]  #全部调出
    df["cleaned"] = mask  #记录是否导致了缺码以及清零，用于之后的齐码率保护
    return df

df_=group.apply(clean)

# 2.3(库存量保护)  以及2.1(满场率保护)
##在这里我们判断调出后是否满足库存量以及满场率，如果不满足则优先保留调出导致断码的货品（提升满场率），
##并且优先保留调出需求小的（可提升满场率以及库存量）
group = df_.groupby(by = "ShopCode",sort = False)
def MinSkc(df):#满场率的保护
    mask =(((df["Qty"]+df["SkuNeed"])==0)*df["SizeCoreFlag"])==1 #断码的货品
    skc = df["MatCode"][mask] #进而找到缺码的skc的编号
    mask=df["MatCode"].apply(lambda x: False if x in skc.values else True) #找出所有齐码的
    skc_num = df["MatCode"][mask].value_counts().size
    diff=int(skc_num-df["SkcNum"].mean())
    if diff<0: #如果不满足满场率
        if(df["MatCode"][df["cleaned"]].value_counts().size<=(-diff)):
            df["SkuNeed"][df["cleaned"]]=0
            df["cleaned"]=False #因断码而被清空的货品全部用来恢复满场率
        else:
            lst=df[df["SkuNeed"]<0].groupby(by = "MatCode",sort = False)["SkuNeed"].sum().sort_values(ascending = False).index[:-diff]
            mask =df["MatCode"].apply(lambda x: True if x in lst.values else False)#挑选出相应skc
            df["SkuNeed"][mask] = 0
            df["cleaned"][mask]=False
            
    return df    
df_ = group.apply(MinSkc) 
group = df_.groupby(by = "ShopCode",sort = False)
def MinStorageProtect(df):
    store =(df["Qty"]+df["SkuNeed"] ).sum()#库存
    diff =int( store - df["StorageMin"].mean())
    if diff<0: #如果不满足最低库存，则优先保留调出需求最大的，因为之前经过了满场率的清洗，所以满场率不重要，库存重要
        df_temp = df[df["SkuNeed"]<0].sort_values(by = "SkuNeed",ascending = True)
        sum_ = 0
        for ind in df_temp.index.values:
            sum_+=abs(df["SkuNeed"][ind])
            df["SkuNeed"][ind] =0
            if (sum_+diff)>=0:
                break
        mask =(((df["Qty"]+df["SkuNeed"])==0)*df["SizeCoreFlag"]*(df["SkuNeed"]<0)==1) #找到导致缺码且要调出的sku
        df["cleaned"] = mask  #更新是否导致了缺码以及清零，避免出现错误
        return df 
df_=group.apply(MinStorageProtect)
            
               
           
        
           
           
           
    
     









/home/gang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [47]:
df_final = df_.reset_index(drop = True)

In [50]:
df_final.head()

,AreaName,ShopCode,MatCode,SizeName,LastWeekSaleQty,AvgLastWeekSaleQty,LastWeekMoveQty,LastWeekInitQty,CZPFlag,SKCCzpFlag,...,店铺名称,店铺分档,店铺类型,SkcNum,满场率标准,UniformRate,动销率标准,库存均深标准,StorageMin,cleaned
0,朔州市,01023,3B60001B2,JM,1,0.14,0.0,4,平,平,...,朔州北关一店,重点店,E2,143,90%-120%,0.761905,0.439235,11.476063,1477,False
1,朔州市,01023,3C10001W0,JM,0,0.00,0.0,47,超滞,超滞,...,朔州北关一店,重点店,E2,143,90%-120%,0.761905,0.439235,11.476063,1477,False
2,朔州市,01023,3J10002B0,JM,0,0.00,0.0,19,超滞,超滞,...,朔州北关一店,重点店,E2,143,90%-120%,0.761905,0.439235,11.476063,1477,False
3,朔州市,01023,3J10003S0,JM,1,0.14,0.0,1,爆,爆,...,朔州北关一店,重点店,E2,143,90%-120%,0.761905,0.439235,11.476063,1477,False
4,朔州市,01023,3JP11001R3,M,0,0.00,0.0,0,超滞,超滞,...,朔州北关一店,重点店,E2,143,90%-120%,0.761905,0.439235,11.476063,1477,False


In [51]:
df_final.columns

Index(['AreaName', 'ShopCode', 'MatCode', 'SizeName', 'LastWeekSaleQty',
       'AvgLastWeekSaleQty', 'LastWeekMoveQty', 'LastWeekInitQty', 'CZPFlag',
       'SKCCzpFlag', 'CZPSuppleRate', 'DisPlayQty', 'SizeCoreFlag', 'Qty',
       'SkuNeed', 'need_coefficient', '分公司', '店铺名称', '店铺分档', '店铺类型', 'SkcNum',
       '满场率标准', 'UniformRate', '动销率标准', '库存均深标准', 'StorageMin', 'cleaned'],
      dtype='object')

In [ ]:
df_final.to_csv("")